In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


In [2]:
output_data_path = "../WeatherPy/output_data.csv"

# Import the output data.csv file as a DataFrame
weather_df = pd.read_csv(output_data_path)
weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Port Elizabeth,-33.92,25.57,68.00,82,0,13.87,ZA,1608059557
1,1,Cabo San Lucas,22.89,-109.91,75.00,60,20,3.36,MX,1608059391
2,2,Lebu,-37.62,-73.65,56.53,59,0,23.44,CL,1608059429
3,3,Bredasdorp,-34.53,20.04,60.80,67,0,3.36,ZA,1608059558
4,4,Noshiro,40.20,140.02,28.96,89,99,21.16,JP,1608059558
...,...,...,...,...,...,...,...,...,...,...
541,541,Mogadishu,2.04,45.34,77.56,79,0,19.86,SO,1608059647
542,542,Ulladulla,-35.35,150.47,70.00,100,90,9.17,AU,1608059647
543,543,Lubumbashi,-11.66,27.48,64.47,94,100,4.21,CD,1608059647
544,544,Manggar,-2.88,108.27,80.15,81,100,4.05,ID,1608059647


In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Create a map using lat & lng as locations
marker_locations = weather_df[['Lat', 'Lng']].astype(float)

# Convert humidity to float and store

humidity = weather_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(marker_locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

plt.savefig("figure1.png")

fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [5]:
#Create new Dataframe with weather criteria
#Max temp <80 but >70
#Wind speed less than 10mph
#Zero cloudiness

new_data_df = weather_df.loc[(weather_df['Max Temp'] <= 80) & 
                            (weather_df['Max Temp'] > 60) &
                            (weather_df["Wind Speed"] < 10) &
                            (weather_df["Cloudiness"] == 0)]
new_data_df = new_data_df.dropna(how='any')
new_data_df = pd.DataFrame(new_data_df, columns=["City", "Max Temp", "Wind Speed", "Cloudiness"])
new_data_df

,City,Max Temp,Wind Speed,Cloudiness
3,Bredasdorp,60.80,3.36,0
26,Hermanus,61.00,7.72,0
32,Port Alfred,66.99,4.00,0
59,Ahipara,61.88,6.24,0
81,Atar,73.24,9.78,0
87,Gisborne,64.99,3.00,0
92,Aswān,61.03,8.75,0
108,Salalah,73.40,5.82,0
194,Mildura,71.60,9.17,0
210,Marsabit,68.61,6.60,0


In [6]:
#Create a column in the dataframe for the hotel name
hotel_df = pd.DataFrame(weather_df)
hotel_df["Hotel Name"] = " "
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,Port Elizabeth,-33.92,25.57,68.00,82,0,13.87,ZA,1608059557,
1,1,Cabo San Lucas,22.89,-109.91,75.00,60,20,3.36,MX,1608059391,
2,2,Lebu,-37.62,-73.65,56.53,59,0,23.44,CL,1608059429,
3,3,Bredasdorp,-34.53,20.04,60.80,67,0,3.36,ZA,1608059558,
4,4,Noshiro,40.20,140.02,28.96,89,99,21.16,JP,1608059558,


In [16]:
#Set parameters to search for hotels within 5000 meters

target_coordinates = f"{hotel_df['Lat'][0]}, {hotel_df['Lng'][0]}"
target_search = "hotel" 
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params).json()
print(json.dumps(response, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "next_page_token": "ATtYBwLJvlIjXX-aR2SZteOIayQv6BWD5yQciegK5Oi3RLAOXdCU_KhssaPGR0L0o-q8FXjzwQXATNZNF4H0JfJ4qezVOnO1KowZ6IVu_lIJw0R7guyH8HPzN1FKYRWIs5fDOxQ_Xg7Q5LfwWOd0Z_mvdli2Gyp8CEbB3GfZ84Iga6-iqfNlUf0zAaCBc21zU9Dyll8tfWpXVZPNXzSxeNn3y29nLudwLiiHTT0lkjM7bHuu9SDyUc5sWWOMejxnDUja_E1pjoJlYlLuhDYGcPgctZdpdlzdHcqQ6VS8GOHf8XHekBFdRVyvwWQEc6be6XBTDFzBRGBHtLDsbFlnGlWW8EppipfTU6fqnopc0rU4_v8PTj6MZ5goI3PhdaADuCUfwVd-G4EehuHK0WjDEUvgt9g2qngGTwBWgTgVGviTEH0ruARaYCrjuTvmLoXWuQU",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -33.963425,
                    "lng": 25.6197883
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.96217257010728,
                        "lng": 25.62117297989272
                    },
                    "southwest": {
                        "lat": -33.96487222

In [46]:
#create an empty list for the hotel names
hotel_name = []

#set up the parameters dictonary
params = {
    "radius":5000,
    "types": "hotel",
    "keyword":"hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat}, {lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    
    hotel_response = requests.get(base_url, params=params).json()
    
    try:
    
        hotel_name.append(hotel_response["results"][0]["name"])
    
    except:
        hotel_name.append(np.nan)

In [47]:
hotel_df["Hotel Name"] = hotel_name
hotel_df.dropna(how='any')
hotel_df.to_csv("hotel_listings.csv")

In [48]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [49]:
 # Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
plt.savefig("figure2.png")
fig

#Display figure

Figure(layout=FigureLayout(height='420px'))